In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.1'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

In [ ]:
plus_20_votes_df = vine_df.filter(col("total_votes")>20)
plus_20_votes_df.show()

In [ ]:
# Filter dataframe for records with >50% helpful votes to total votes
plus_50_pct_df = plus_20_votes_df.filter((col("helpful_votes")/col("total_votes"))>=.5)
plus_50_pct_df.show()

In [ ]:
# Filter dataframe for reviews written as part of the Vine program 
vine_df = plus_50_pct_df.filter(col("vine")=="Y")
vine_df.show()

In [ ]:
# Filter dataframe for reviews written as part of the Vine program 
not_vine_df = plus_50_pct_df.filter(col("vine")=="N")
not_vine_df.show()

In [ ]:
# Determine total number of reviews
total_reviews = plus_50_pct_df.count()
total_reviews

In [ ]:
# Determine the total 5 star ratings
# where method syntax found at https://www.geeksforgeeks.org/count-values-by-condition-in-pyspark-dataframe/
total_5_star = plus_50_pct_df.select('star_rating').where(plus_50_pct_df.star_rating==5).count()
total_5_star

In [ ]:
# Determine total percentage of 5 start reviews
total_prcnt_5_star = (total_5_star/total_reviews)*100
total_prcnt_5_star

In [ ]:
# Determine number of paid reviews
number_paid = vine_df.count()
number_paid

In [ ]:
31
# Determine the number 5 star paid reviews 
paid_5_star = vine_df.select('star_rating').where(vine_df.star_rating==5).count()
paid_5_star

In [ ]:
# Determine percentage of paid 5 star reviews
prcnt_5_star_paid = (paid_5_star/number_paid)*100
prcnt_5_star_paid

In [ ]:
# Determine number of unpaid reviews
number_unpaid = not_vine_df.count()
number_unpaid

In [ ]:
# Determine the number 5 star unpaid reviews 
unpaid_5_star = not_vine_df.select('star_rating').where(not_vine_df.star_rating==5).count()
unpaid_5_star

In [ ]:
# Determine percentage of unpaid 5 star reviews
prcnt_5_star_unpaid = (unpaid_5_star/number_unpaid)*100
prcnt_5_star_unpaid